In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
import time
import matplotlib.pyplot as plt


import requests
from datetime import datetime

# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

In [3]:
import random

In [4]:
# Importing cities csv file
file_path = os.path.join("weather_data","cities.csv")
city_data_df = pd.read_csv(file_path)
city_data_df

,City_ID,City,Date,Lat,Lng,Max Temp,Humidity,Country,Cloudiness,Wind Speed
0,0,Port Elizabeth,2022-03-25 01:11:07,-33.9180,25.5701,64.09,99,ZA,100,10.36
1,1,Lagos,2022-03-25 01:07:29,6.5833,3.7500,81.36,83,NG,100,6.22
2,2,Severo-Kurilsk,2022-03-25 01:11:08,50.6789,156.1250,30.15,67,RU,99,9.95
3,3,Kyabe,2022-03-25 01:11:08,9.4515,18.9449,84.02,40,TD,83,2.30
4,4,Busselton,2022-03-25 01:11:09,-33.6500,115.3333,72.00,41,AU,36,12.84
...,...,...,...,...,...,...,...,...,...,...
572,572,Sanmenxia,2022-03-25 01:27:14,34.7747,111.1814,56.52,66,CN,100,9.51
573,573,Aljezur,2022-03-25 01:27:15,37.3191,-8.8033,51.26,90,PT,79,6.06
574,574,Rocha,2022-03-25 01:27:15,-34.4833,-54.3333,57.31,59,UY,93,8.59
575,575,Martapura,2022-03-25 01:27:17,-3.4167,114.8500,77.09,94,ID,20,0.00


In [5]:
# Creating a new set of 2,000 random latitudes and logitudes

# Creating 2,000 random lats and lngs
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)

In [6]:
# Creating the 2,000 sets of lats and lngs
lat_lngs = zip(lats,lngs)

# Using the citipy module - passing in a set of lats and lngs, will determine the closest city and its country code

# Creating lists to contain the city names and country codes
cities = []
country_codes = []

# Looping through the different tuples () and passing the contents into citipy's nearest_city function
# adding the city name and country code
for cords in lat_lngs:
    cities.append(citipy.nearest_city(cords[0], cords[1]).city_name)
    country_codes.append(citipy.nearest_city(cords[0], cords[1]).country_code)

In [7]:
# Using OpenWeatherMaps API to retrieve the different cities weather information

# Importing the openweathers api key to use it securely
from config import weather_api_key

In [8]:
# URL to make an API request

# https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={API key}
base_url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key + "&q="
base_url

'http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=e52763fce3c9071965e25df476735e27&q='

In [9]:
# EXAMPLE

# Replace function ensures there is no spaces in the url - replaces all spaces with "+"
base_url + cities[0].replace(" ", "+")

# Use the requests function to request data from the specific API server
requests.get(base_url + cities[0]).json()

{'coord': {'lon': -175.2, 'lat': -21.2},
 'weather': [{'id': 801,
   'main': 'Clouds',
   'description': 'few clouds',
   'icon': '02n'}],
 'base': 'stations',
 'main': {'temp': 77.16,
  'feels_like': 78.48,
  'temp_min': 77.16,
  'temp_max': 77.16,
  'pressure': 1011,
  'humidity': 83},
 'visibility': 10000,
 'wind': {'speed': 8.05, 'deg': 160},
 'clouds': {'all': 20},
 'dt': 1648542218,
 'sys': {'type': 1,
  'id': 7285,
  'country': 'TO',
  'sunrise': 1648576036,
  'sunset': 1648619003},
 'timezone': 46800,
 'id': 4032243,
 'name': 'Vaini',
 'cod': 200}

In [10]:
# Looping through an enumerated cities list, requesting data from the API, and appending them to a city data list

# Create an empty list to hold the weather data.
city_data = []

# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

for i, city in enumerate(cities):
    # Setting up the correct API city request url - Replace function all white space with a "+"
    city_url = base_url + city.replace(" ", "+")
    
    # Group cities in sets of 50 for logging purposes - Can only make 60 calls from the API per minute
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)
        
    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
        
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        
        # Navigating through the recieved json doc and retrieving the target data
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description = city_weather['weather'][0]['description']
        
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        
         # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "% Humidity": city_humidity,
                          "% Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date,
                         "Weather description": city_description})

    # If an error is experienced, skip the city.
    except:
        print("City: " +city+ " not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | vaini
Processing Record 2 of Set 1 | rangoon
Processing Record 3 of Set 1 | rikitea
Processing Record 4 of Set 1 | rikitea
Processing Record 5 of Set 1 | pisco
Processing Record 6 of Set 1 | ribeira grande
Processing Record 7 of Set 1 | coquimbo
Processing Record 8 of Set 1 | karoi
Processing Record 9 of Set 1 | mafra
Processing Record 10 of Set 1 | nikolskoye
Processing Record 11 of Set 1 | karkaralinsk
City: karkaralinsk not found. Skipping...
Processing Record 12 of Set 1 | port elizabeth
Processing Record 13 of Set 1 | puerto ayora
Processing Record 14 of Set 1 | broken hill
Processing Record 15 of Set 1 | cayenne
Processing Record 16 of Set 1 | rikitea
Processing Record 17 of Set 1 | mullaitivu
City: mullaitivu not found. Skipping...
Processing Record 18 of Set 1 | klaksvik
Processing Record 19 of Set 1 | male
Processing Record 20 of Set 1 | rikitea
Processing Record 21 of Set 1 | esperance


Processing Record 32 of Set 4 | humaita
Processing Record 33 of Set 4 | koboldo
City: koboldo not found. Skipping...
Processing Record 34 of Set 4 | vao
Processing Record 35 of Set 4 | wawa
Processing Record 36 of Set 4 | garissa
Processing Record 37 of Set 4 | cape town
Processing Record 38 of Set 4 | banda aceh
Processing Record 39 of Set 4 | gazanjyk
Processing Record 40 of Set 4 | kapaa
Processing Record 41 of Set 4 | new norfolk
Processing Record 42 of Set 4 | burdur
Processing Record 43 of Set 4 | inhambane
Processing Record 44 of Set 4 | norman wells
Processing Record 45 of Set 4 | hermanus
Processing Record 46 of Set 4 | new norfolk
Processing Record 47 of Set 4 | sedelnikovo
City: sedelnikovo not found. Skipping...
Processing Record 48 of Set 4 | rikitea
Processing Record 49 of Set 4 | cap malheureux
Processing Record 50 of Set 4 | new norfolk
Processing Record 1 of Set 5 | east london
Processing Record 2 of Set 5 | aberdeen
Processing Record 3 of Set 5 | zhigansk
Processing R

City: amderma not found. Skipping...
Processing Record 16 of Set 8 | emet
Processing Record 17 of Set 8 | kodiak
Processing Record 18 of Set 8 | noumea
Processing Record 19 of Set 8 | sitio novo do tocantins
Processing Record 20 of Set 8 | umm lajj
Processing Record 21 of Set 8 | punta arenas
Processing Record 22 of Set 8 | arraial do cabo
Processing Record 23 of Set 8 | grand baie
Processing Record 24 of Set 8 | cascais
Processing Record 25 of Set 8 | carnarvon
Processing Record 26 of Set 8 | saint-philippe
Processing Record 27 of Set 8 | vaini
Processing Record 28 of Set 8 | plainview
Processing Record 29 of Set 8 | lamu
Processing Record 30 of Set 8 | rikitea
Processing Record 31 of Set 8 | dikson
Processing Record 32 of Set 8 | conceicao do araguaia
Processing Record 33 of Set 8 | vanimo
Processing Record 34 of Set 8 | tres arroyos
Processing Record 35 of Set 8 | guhagar
Processing Record 36 of Set 8 | adrar
Processing Record 37 of Set 8 | maniitsoq
Processing Record 38 of Set 8 | 

Processing Record 44 of Set 11 | rikitea
Processing Record 45 of Set 11 | rio grande
Processing Record 46 of Set 11 | ushuaia
Processing Record 47 of Set 11 | provideniya
Processing Record 48 of Set 11 | kuche
City: kuche not found. Skipping...
Processing Record 49 of Set 11 | bulgan
Processing Record 50 of Set 11 | cape town
Processing Record 1 of Set 12 | barentsburg
City: barentsburg not found. Skipping...
Processing Record 2 of Set 12 | candelaria
Processing Record 3 of Set 12 | illoqqortoormiut
City: illoqqortoormiut not found. Skipping...
Processing Record 4 of Set 12 | tidore
City: tidore not found. Skipping...
Processing Record 5 of Set 12 | cerro largo
Processing Record 6 of Set 12 | puerto ayora
Processing Record 7 of Set 12 | attawapiskat
City: attawapiskat not found. Skipping...
Processing Record 8 of Set 12 | victoria
Processing Record 9 of Set 12 | tacuati
Processing Record 10 of Set 12 | lata
Processing Record 11 of Set 12 | tsihombe
City: tsihombe not found. Skipping...

Processing Record 26 of Set 15 | atuona
Processing Record 27 of Set 15 | mataura
Processing Record 28 of Set 15 | port elizabeth
Processing Record 29 of Set 15 | bay city
Processing Record 30 of Set 15 | qaanaaq
Processing Record 31 of Set 15 | bathsheba
Processing Record 32 of Set 15 | katsuura
Processing Record 33 of Set 15 | klaksvik
Processing Record 34 of Set 15 | mataura
Processing Record 35 of Set 15 | constitucion
Processing Record 36 of Set 15 | taolanaro
City: taolanaro not found. Skipping...
Processing Record 37 of Set 15 | pisco
Processing Record 38 of Set 15 | vaini
Processing Record 39 of Set 15 | cabedelo
Processing Record 40 of Set 15 | kaitangata
Processing Record 41 of Set 15 | castro
Processing Record 42 of Set 15 | longyearbyen
Processing Record 43 of Set 15 | sechura
Processing Record 44 of Set 15 | yarada
Processing Record 45 of Set 15 | rikitea
Processing Record 46 of Set 15 | chinhoyi
Processing Record 47 of Set 15 | illoqqortoormiut
City: illoqqortoormiut not f

...

Processing Record 10 of Set 26 | vila franca do campo
Processing Record 11 of Set 26 | barrow
Processing Record 12 of Set 26 | mataura
Processing Record 13 of Set 26 | bluff
Processing Record 14 of Set 26 | mount isa
Processing Record 15 of Set 26 | berlevag
Processing Record 16 of Set 26 | jamestown
Processing Record 17 of Set 26 | cidreira
Processing Record 18 of Set 26 | ancud
Processing Record 19 of Set 26 | albany
Processing Record 20 of Set 26 | yellowknife
Processing Record 21 of Set 26 | tucuma
Processing Record 22 of Set 26 | port hedland
Processing Record 23 of Set 26 | panjakent
Processing Record 24 of Set 26 | mataura
Processing Record 25 of Set 26 | provideniya
Processing Record 26 of Set 26 | iqaluit
Processing Record 27 of Set 26 | mercedes
Processing Record 28 of Set 26 | ashland
Processing Record 29 of Set 26 | castro
Processing Record 30 of Set 26 | taolanaro
City: taolanaro not found. Skipping...
Processing Record 31 of Set 26 | djenne
Processing Record 32 of Set 26 

Processing Record 43 of Set 29 | carnarvon
Processing Record 44 of Set 29 | jamestown
Processing Record 45 of Set 29 | lac-megantic
Processing Record 46 of Set 29 | new norfolk
Processing Record 47 of Set 29 | mataura
Processing Record 48 of Set 29 | naze
Processing Record 49 of Set 29 | busselton
Processing Record 50 of Set 29 | port alfred
Processing Record 1 of Set 30 | trujillo
Processing Record 2 of Set 30 | kirakira
Processing Record 3 of Set 30 | north myrtle beach
Processing Record 4 of Set 30 | pochutla
Processing Record 5 of Set 30 | oktyabrskiy
Processing Record 6 of Set 30 | tres arroyos
Processing Record 7 of Set 30 | bredasdorp
Processing Record 8 of Set 30 | rikitea
Processing Record 9 of Set 30 | rikitea
Processing Record 10 of Set 30 | avarua
Processing Record 11 of Set 30 | nikolayevsk-na-amure
Processing Record 12 of Set 30 | huarmey
Processing Record 13 of Set 30 | atuona
Processing Record 14 of Set 30 | rikitea
Processing Record 15 of Set 30 | chokurdakh
Processing

Processing Record 27 of Set 33 | vaini
Processing Record 28 of Set 33 | kapaa
Processing Record 29 of Set 33 | bograd
Processing Record 30 of Set 33 | pelym
Processing Record 31 of Set 33 | linjiang
Processing Record 32 of Set 33 | saleaula
City: saleaula not found. Skipping...
Processing Record 33 of Set 33 | brigham city
Processing Record 34 of Set 33 | hokitika
Processing Record 35 of Set 33 | mys shmidta
City: mys shmidta not found. Skipping...
Processing Record 36 of Set 33 | cape town
Processing Record 37 of Set 33 | grodzisk wielkopolski
Processing Record 38 of Set 33 | albany
Processing Record 39 of Set 33 | pokhara
Processing Record 40 of Set 33 | cape town
Processing Record 41 of Set 33 | kamina
Processing Record 42 of Set 33 | ilebo
Processing Record 43 of Set 33 | pulandian
Processing Record 44 of Set 33 | qaanaaq
Processing Record 45 of Set 33 | port alfred
Processing Record 46 of Set 33 | hithadhoo
Processing Record 47 of Set 33 | rikitea
Processing Record 48 of Set 33 | 

Processing Record 3 of Set 37 | barra patuca
Processing Record 4 of Set 37 | rikitea
Processing Record 5 of Set 37 | novosergiyevka
Processing Record 6 of Set 37 | lavrentiya
Processing Record 7 of Set 37 | saint-philippe
Processing Record 8 of Set 37 | aklavik
Processing Record 9 of Set 37 | kholm-zhirkovskiy
Processing Record 10 of Set 37 | takoradi
Processing Record 11 of Set 37 | castro
Processing Record 12 of Set 37 | upernavik
Processing Record 13 of Set 37 | asau
Processing Record 14 of Set 37 | busselton
Processing Record 15 of Set 37 | bredasdorp
Processing Record 16 of Set 37 | vaini
Processing Record 17 of Set 37 | ushuaia
Processing Record 18 of Set 37 | cidreira
Processing Record 19 of Set 37 | opuwo
Processing Record 20 of Set 37 | gold coast
Processing Record 21 of Set 37 | vaini
Processing Record 22 of Set 37 | faanui
Processing Record 23 of Set 37 | tasiilaq
Processing Record 24 of Set 37 | sobolevo
Processing Record 25 of Set 37 | saint-philippe
Processing Record 26 o

Processing Record 34 of Set 40 | hobart
Processing Record 35 of Set 40 | mullaitivu
City: mullaitivu not found. Skipping...
Processing Record 36 of Set 40 | busselton
Processing Record 37 of Set 40 | touros
Processing Record 38 of Set 40 | bredasdorp
Processing Record 39 of Set 40 | butaritari
Processing Record 40 of Set 40 | rikitea
Processing Record 41 of Set 40 | east london
Processing Record 42 of Set 40 | klaksvik
Processing Record 43 of Set 40 | mwinilunga
Processing Record 44 of Set 40 | nikolskoye
Processing Record 45 of Set 40 | darnah
Processing Record 46 of Set 40 | gladstone
Processing Record 47 of Set 40 | itarema
Processing Record 48 of Set 40 | ozinki
Processing Record 49 of Set 40 | new norfolk
Processing Record 50 of Set 40 | marcona
City: marcona not found. Skipping...
-----------------------------
Data Retrieval Complete      
-----------------------------


In [11]:
# Creating a dataframe with the city data ive recieved

weather_data_df = pd.DataFrame(city_data)

In [12]:
weather_data_df

,City,Lat,Lng,Max Temp,% Humidity,% Cloudiness,Wind Speed,Country,Date,Weather description
0,Vaini,-21.2000,-175.2000,77.16,83,20,8.05,TO,2022-03-29 08:23:38,few clouds
1,Rangoon,16.8053,96.1561,94.96,52,75,6.91,MM,2022-03-29 08:23:39,broken clouds
2,Rikitea,-23.1203,-134.9692,77.81,70,92,2.71,PF,2022-03-29 08:23:40,overcast clouds
3,Rikitea,-23.1203,-134.9692,77.81,70,92,2.71,PF,2022-03-29 08:23:40,overcast clouds
4,Pisco,-13.7000,-76.2167,66.25,82,40,13.80,PE,2022-03-29 08:23:41,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
1818,Darnah,32.7670,22.6367,65.12,50,0,9.44,LY,2022-03-29 09:13:37,clear sky
1819,Gladstone,-23.8500,151.2500,82.33,39,19,4.61,AU,2022-03-29 09:13:37,few clouds
1820,Itarema,-2.9248,-39.9167,74.44,95,92,7.67,BR,2022-03-29 09:13:38,overcast clouds
1821,Ozinki,51.1786,49.6769,35.13,98,86,23.98,RU,2022-03-29 09:13:38,overcast clouds


In [13]:
# Create the output file (CSV).
output_data_file = "weather_data/WeatherPy_Database.csv"
# Export the City_Data into a CSV.
weather_data_df.to_csv(output_data_file, index_label="City_ID")